In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import xlrd, csv
import datetime as dt 

In [3]:
leads = pd.read_csv('Input4.csv', encoding = "ISO-8859-1")

/Users/hariniashok/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##### Calculating Leads and Sales for all fields

In [4]:
#lead value
leads['Lead'] = 1
#Sale value
leads['Sale']  = 0
leads['Sale'][leads['Stage'] == 'Closed Won'] = 1
leads['fix'] = 0
leads['met'] = 1
cond = ['Met and interested', 'Met and Not Interested', 'Not Qualified', 'Met but not interested', 'Appointment fixed', 'Booking Confirmed']

for c in cond:
    leads['fix'][leads['Lead Enquiry Stage'] == c] = 1
    if c == 'Appointment fixed' or c == 'Booking Confirmed':
        leads['met'][leads['Lead Enquiry Stage'] == c] = 0
    else:
        leads['met'][leads['Lead Enquiry Stage'] == c] = 1

/Users/hariniashok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/hariniashok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/hariniashok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

Noise removal - Removing low contributing fields and outliers

In [6]:
leads = leads[leads['Campaign Type'] != 'National Alliance']
leads = leads[leads['Ethnicity'] != 'north east']
leads = leads[leads['Car Size'] != 'audi']
leads = leads[leads['Car Size'] != 'datsun']
leads = leads[leads['Car Size'] != 'jeep']


#### Initial Target Sales %

In [7]:
target = len(leads[leads['Sale']  == 1])/len(leads)
target*100

0.15407917840519506

In [7]:
leads['Sale'].sum()/leads['Lead'].count()*100

0.15341188327153207

In [8]:
leads['Sale'].sum()/len(leads['Lead'])*100

0.15341188327153207

##### Data Cleaning - correcting field categories

In [25]:
leads['Derived Age Group'].replace(to_replace ="42", value ="40 to 49")

0             36 to 39
1          age unknown
2          age unknown
3          age unknown
4          age unknown
              ...     
2008314    age unknown
2008315    age unknown
2008316    age unknown
2008317    age unknown
2008318    age unknown
Name: Derived Age Group, Length: 2008319, dtype: object

In [28]:
tmp['Derived Age Group'].replace(to_replace ="42", value ="40 to 49")

0             36 to 39
1          age unknown
2          age unknown
3          age unknown
4          age unknown
              ...     
2008314            NaN
2008315            NaN
2008316            NaN
2008317            NaN
2008318            NaN
Name: Derived Age Group, Length: 2008319, dtype: object

In [47]:
leads = leads.replace(to_replace ="ethnicity_unknown", value ="ethnicity unknown")
leads['Ethnicity'].fillna('ethnicity unknown', inplace=True)

leads['Tier'].fillna('Tier Unknown', inplace=True)

leads['Car Size'] = leads['Car Size'].replace(to_replace ="yes", value ="size unknown")
leads['Car Size'].fillna('size unknown', inplace=True)

leads['Derived Age Group'] = leads['Derived Age Group'].replace(to_replace ="42", value ="40 to 49")
leads['Derived Age Group'].fillna('age unknown', inplace=True)

leads['Derived Children'].fillna('child unknown', inplace=True)

leads['Derived Marital Status'].fillna('marital unknown', inplace=True)
leads['Derived Marital Status'] = leads['Derived Marital Status'].replace(to_replace ="MARRIED", value ="married")
leads['Derived Marital Status'] = leads['Derived Marital Status'].replace(to_replace ="singal", value ="single")

leads['Car Segment'] = leads['Car Segment'].replace(to_replace ="yes", value ="segment unknown")
leads['Car Segment'].fillna('segment unknown', inplace=True)

leads['Derived Credit Card'] = leads['Derived Credit Card'].replace(to_replace ="YES", value ="cc unknown")
leads['Derived Credit Card'] = leads['Derived Credit Card'].replace(to_replace ="credit card unknown", value ="cc unknown")
leads['Derived Credit Card'].fillna('cc unknown', inplace=True)

leads['Derived Income Range'] = leads['Derived Income Range'].replace(to_replace ="70K", value ="rs.65000 to rs.100000")
leads['Derived Income Range'].fillna('income unknown', inplace=True)


leads['Email Domain'].fillna('email unknown', inplace=True)
leads['Email Domain'] = leads['Email Domain'].replace(to_replace ="mnazim_shaikh@rediffmail.com", value ="rediff.com")
leads['Email Domain'] = leads['Email Domain'].replace(to_replace ="paragjani@gmail.com", value ="gmail.com")



In [13]:
ldf['new_date'] = pd.to_datetime(ldf['Created Date'])

In [18]:
ldf2 = ldf

Dropping non-feature columns

In [19]:
ldf2 = ldf2.drop(['Lead ID', 'Rating', 'Created Date', '18 Digit',
                                         'fix', 'met', 'new_date', 'Mobile', 'Stage','Lead Enquiry Stage'], axis=1)

In [8]:
ldf.columns

Index(['Lead ID', 'Ethnicity', 'Mobile Operator', 'Email Domain',
       'Campaign Type', 'Car Segment', 'Derived Occupation',
       'Derived Marital Status', 'Derived Children', 'Derived Age Group',
       'Derived Credit Card', 'Derived Income Range', 'Car Size', 'Tier',
       'Branch Master', 'City', 'Branch', 'Rating', 'Mobile', 'Stage',
       'Lead Enquiry Stage', 'Created Date', 'Campaign City',
       'Campaign Category', 'Count Activity', '18 Digit', 'Lead', 'Sale',
       'fix', 'met'],
      dtype='object')

groupby month and year

In [35]:
by_mon = ldf.groupby([ldf['new_date'].dt.month, ldf['new_date'].dt.year])['Sale']
by_mon.agg(lambda x:  (x.eq(1).sum()/len(ldf)))

new_date  new_date
1.0       2019.0      0.000048
          2020.0      0.000001
2.0       2019.0      0.000038
          2020.0      0.000005
3.0       2019.0      0.000046
          2020.0      0.000003
4.0       2019.0      0.000129
          2020.0      0.000003
5.0       2019.0      0.000146
          2020.0      0.000007
6.0       2019.0      0.000145
7.0       2019.0      0.000159
8.0       2019.0      0.000162
9.0       2019.0      0.000185
10.0      2019.0      0.000146
11.0      2019.0      0.000168
12.0      2019.0      0.000140
Name: Sale, dtype: float64

#### Grouping into Monthly Cycles (MCs)

In [9]:
ldf = ldf.drop(['Lead ID', 'Rating', 'Created Date', '18 Digit',
                'fix', 'met', 'Mobile', 'Stage','Lead Enquiry Stage', 'City'], axis=1)

In [10]:
ldf.columns

Index(['Ethnicity', 'Mobile Operator', 'Email Domain', 'Campaign Type',
       'Car Segment', 'Derived Occupation', 'Derived Marital Status',
       'Derived Children', 'Derived Age Group', 'Derived Credit Card',
       'Derived Income Range', 'Car Size', 'Tier', 'Branch Master', 'Branch',
       'Campaign City', 'Campaign Category', 'Count Activity', 'Lead', 'Sale'],
      dtype='object')

In [11]:
ldf.to_csv('ldf.csv')

In [38]:
mc = {}
for i in range(8):
    d1 = int(input("day 1: "))
    mon1 = int(input("month 1 : "))
    
    d2 = int(input("day 2: "))
    mon2 = int(input("month 2 : "))
    mc1_1 = ldf[ldf['new_date'].dt.date >= dt.date(2019, mon1, d1)]['new_date']
    mc1_2 = mc1_1[mc1_1.dt.date <= dt.date(2019, mon2, d2)]
    mc[i+1] = ldf.loc[mc1_2.index]

day 1: 1
month 1 : 4
day 2: 28
month 2 : 4
day 1: 29
month 1 : 4
day 2: 26
month 2 : 5
day 1: 27
month 1 : 5
day 2: 23
month 2 : 6
day 1: 24
month 1 : 6
day 2: 21
month 2 : 7
day 1: 22
month 1 : 7
day 2: 18
month 2 : 8
day 1: 19
month 1 : 8
day 2: 15
month 2 : 9
day 1: 16
month 1 : 9
day 2: 13
month 2 : 10
day 1: 14
month 1 : 10
day 2: 10
month 2 : 11


In [ ]:
mc2 = {}
for i in range(8):
    d1 = int(input("day 1: "))
    mon1 = int(input("month 1 : "))
    
    d2 = int(input("day 2: "))
    mon2 = int(input("month 2 : "))
    mc1_1 = ldf[ldf['new_date'].dt.date >= dt.date(2020, mon1, d1)]['new_date']
    mc1_2 = mc1_1[mc1_1.dt.date <= dt.date(2020, mon2, d2)]
    mc2[i+1] = ldf.loc[mc1_2.index]

In [ ]:
moncyc = {}
for i in range(8):
    moncyc[i+1] = mc[i+1].append(mc2[i+1])

#### Grouping by fields

In [ ]:
campaign = pd.DataFrame(data=camp_cat.agg(lambda x:  (x.eq(1).sum()/len(ldf))*100))

camp_cat.agg(lambda x:  (x.eq(1).sum()/len(ldf))*100 )

In [20]:
fields = list(ldf.columns)
fields = [e for e in fields if e not in ('Lead ID', 'Rating', 'Created Date', '18 Digit', 'Lead', 'Sale', 
                                         'fix', 'met', 'new_date', 'Mobile', 'Stage','Lead Enquiry Stage'
                                        )]

In [21]:
fields

['Ethnicity',
 'Mobile Operator',
 'Email Domain',
 'Campaign Type',
 'Car Segment',
 'Derived Occupation',
 'Derived Marital Status',
 'Derived Children',
 'Derived Age Group',
 'Derived Credit Card',
 'Derived Income Range',
 'Car Size',
 'Tier',
 'Branch Master',
 'Branch',
 'Campaign City',
 'Campaign Category',
 'Count Activity']

In [72]:
all_fields = {}
for field in fields:
    for i in range(8): 
        mon_cat = moncyc[i+1].groupby([moncyc[i+1][field]])
        if i == 0:
            mc_cat = pd.DataFrame(data=mon_cat['Sale'].sum()/mon_cat['Lead'].count()*100)
        else:
            mc_cat[str(i+1)] = mon_cat['Sale'].sum()/mon_cat['Lead'].count()*100
    mc_cat = mc_cat.rename(columns={'Sale': '1'})
    all_fields[field] = mc_cat

In [64]:
mon_cat = moncyc[1].groupby([moncyc[1]['Mobile Operator']])

In [73]:
writer = pd.ExcelWriter('percentages_new.xlsx', engine='xlsxwriter')
for i in fields:
    all_fields[i].to_excel(writer, sheet_name=i)
writer.save()

In [15]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

### Feature Selection Chi-squared

In [17]:
from scipy.stats import chi2_contingency


In [14]:
ldf.Lead = ldf.Lead.astype(object)
ldf.Sale = ldf.Sale.astype(object)

In [15]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008319 entries, 0 to 2008318
Data columns (total 20 columns):
Ethnicity                 object
Mobile Operator           object
Email Domain              object
Campaign Type             object
Car Segment               object
Derived Occupation        object
Derived Marital Status    object
Derived Children          object
Derived Age Group         object
Derived Credit Card       object
Derived Income Range      object
Car Size                  object
Tier                      object
Branch Master             object
Branch                    object
Campaign City             object
Campaign Category         object
Count Activity            float64
Lead                      object
Sale                      object
dtypes: float64(1), object(19)
memory usage: 306.4+ MB


In [18]:
chi_res = chi2_contingency(pd.crosstab(ldf['Sale'], ldf['Car Size']))

In [19]:
print('Chi2 Statistic: {}, p-value: {}'.format(chi_res[0], chi_res[1]))

Chi2 Statistic: 342.80973290552583, p-value: 9.306998304408828e-59


In [23]:
chi2_check = []
for i in fields:
    if chi2_contingency(pd.crosstab(ldf['Sale'], ldf[i]))[1] < 0.05:
        chi2_check.append('Reject Null Hypothesis')
    else:
        chi2_check.append('Fail to Reject Null Hypothesis')
res = pd.DataFrame(data = [fields, chi2_check] 
             ).T 
res.columns = ['Column', 'Hypothesis']
print(res)

                    Column              Hypothesis
0                Ethnicity  Reject Null Hypothesis
1          Mobile Operator  Reject Null Hypothesis
2             Email Domain  Reject Null Hypothesis
3            Campaign Type  Reject Null Hypothesis
4              Car Segment  Reject Null Hypothesis
5       Derived Occupation  Reject Null Hypothesis
6   Derived Marital Status  Reject Null Hypothesis
7         Derived Children  Reject Null Hypothesis
8        Derived Age Group  Reject Null Hypothesis
9      Derived Credit Card  Reject Null Hypothesis
10    Derived Income Range  Reject Null Hypothesis
11                Car Size  Reject Null Hypothesis
12                    Tier  Reject Null Hypothesis
13           Branch Master  Reject Null Hypothesis
14                  Branch  Reject Null Hypothesis
15           Campaign City  Reject Null Hypothesis
16       Campaign Category  Reject Null Hypothesis
17          Count Activity  Reject Null Hypothesis


In [29]:
car_size_dummies = pd.get_dummies(data = ldf[['Car Size', 'Sale']], columns = ['Car Size'])

In [30]:
car_size_dummies.columns

Index(['Sale', 'Car Size_audi', 'Car Size_bmw', 'Car Size_chevrolet',
       'Car Size_datsun', 'Car Size_fiat', 'Car Size_ford', 'Car Size_honda',
       'Car Size_hyundai', 'Car Size_jeep', 'Car Size_jlr',
       'Car Size_mahindra', 'Car Size_maruti', 'Car Size_mercedes benz',
       'Car Size_nexa', 'Car Size_nissan', 'Car Size_no car', 'Car Size_other',
       'Car Size_renault', 'Car Size_size unknown', 'Car Size_skoda',
       'Car Size_tata', 'Car Size_toyota', 'Car Size_vw', 'Car Size_yes'],
      dtype='object')

In [72]:
cat = {}
for field in fields:
    dummies = pd.get_dummies(data = ldf[[field, 'Sale']], columns = [field])
    features =  list(dummies.columns)
    features.remove("Sale")
    chi2_check = []
    
    for f in features:
        chi2_check.append(chi2_contingency(pd.crosstab(dummies['Sale'], dummies[f]))[1])
        

    res = pd.DataFrame(data = [features, chi2_check]).T 
    res.columns = ['Feature', 'Hypothesis']
    cat[field] = res


In [73]:
writer = pd.ExcelWriter('p-value.xlsx', engine='xlsxwriter')
for i in fields:
    cat[i].to_excel(writer, sheet_name=i)
writer.save()

### Ratio of features where hyp = 1 to no. of features

In [46]:
cat = {}
for field in fields:
    dummies = pd.get_dummies(data = ldf[[field, 'Sale']], columns = [field])
    features =  list(dummies.columns)
    features.remove("Sale")
    chi2_check = []
    
    for f in features:
        if chi2_contingency(pd.crosstab(dummies['Sale'], dummies[f]))[1] < 0.05:
            chi2_check.append(1)
        else:
            chi2_check.append(0)

    c = chi2_check.count(1)/len(chi2_check)
    cat[field] = c

In [50]:
final = []
for key, item in cat.items():
    if item > 0.5:
        final.append(key)

In [54]:
final.remove('Count Activity')
final.remove('Branch')

### Adjusted p-value

In [63]:
feat = {}
cat = {}
for field in fields:
    dummies = pd.get_dummies(data = ldf[[field, 'Sale']], columns = [field])
    features =  list(dummies.columns)
    features.remove("Sale")
    chi2_check = []
    bon_p_value = 0.05/len(features)
    for f in features:
        if chi2_contingency(pd.crosstab(dummies['Sale'], dummies[f]))[1] < bon_p_value:
            chi2_check.append(1)
        else:
            chi2_check.append(0)

    res = pd.DataFrame(data = [features, chi2_check]).T 
    res.columns = ['Feature', 'Hypothesis']
    c = chi2_check.count(1)/len(chi2_check)
    feat[field] = c
    cat[field] = res

In [64]:
final = []
for key, item in feat.items():
    if item > 0.5:
        final.append(key)

In [66]:
final.remove('Count Activity')
final.remove('Branch')
final

['Campaign Type',
 'Car Segment',
 'Derived Marital Status',
 'Derived Children',
 'Derived Age Group',
 'Derived Credit Card',
 'Derived Income Range',
 'Tier',
 'Branch Master',
 'Campaign Category']

In [67]:
writer = pd.ExcelWriter('features.xlsx', engine='xlsxwriter')
for i in fields:
    cat[i].to_excel(writer, sheet_name=i)
writer.save()

In [68]:
not_selected = [e for e in fields if e not in final]

In [69]:
not_selected

['Ethnicity',
 'Mobile Operator',
 'Email Domain',
 'Derived Occupation',
 'Car Size',
 'Branch',
 'Campaign City',
 'Count Activity']

In [75]:
temp = dict.fromkeys(final, 0)
relation = dict.fromkeys(not_selected, temp)
for no_field in not_selected:
    for in_field in final:
        if chi2_contingency(pd.crosstab(ldf[no_field], ldf[in_field]))[1] < 0.05:
            relation[no_field][in_field] = 1
        else:
            relation[no_field][in_field]= 0


In [76]:
from pprint import pprint

pprint(relation)



{'Branch': {'Branch Master': 1,
            'Campaign Category': 1,
            'Campaign Type': 1,
            'Car Segment': 1,
            'Derived Age Group': 1,
            'Derived Children': 1,
            'Derived Credit Card': 1,
            'Derived Income Range': 1,
            'Derived Marital Status': 1,
            'Tier': 1},
 'Campaign City': {'Branch Master': 1,
                   'Campaign Category': 1,
                   'Campaign Type': 1,
                   'Car Segment': 1,
                   'Derived Age Group': 1,
                   'Derived Children': 1,
                   'Derived Credit Card': 1,
                   'Derived Income Range': 1,
                   'Derived Marital Status': 1,
                   'Tier': 1},
 'Car Size': {'Branch Master': 1,
              'Campaign Category': 1,
              'Campaign Type': 1,
              'Car Segment': 1,
              'Derived Age Group': 1,
              'Derived Children': 1,
              'Derived Credit C

In [49]:
leads['new_date'] = pd.to_datetime(leads['Created Date'])
leads = leads.drop(['Lead ID', 'Rating', 'Created Date', '18 Digit',
                 'Mobile', 'Stage','Lead Enquiry Stage'], axis=1)
with open('input.txt', 'r') as file:
    lines = file.readlines()
mc = {}
m = 1
for i in range(0, len(lines), 4):
    d1 = int(lines[i])
    mon1 = int(lines[i+1])

    d2 = int(lines[i+2])
    mon2 = int(lines[i+3])
    mc1_1 = leads[leads['new_date'].dt.date >= dt.date(2019, mon1, d1)]['new_date']
    mc1_2 = mc1_1[mc1_1.dt.date <= dt.date(2019, mon2, d2)]
    mc[m] = leads.loc[mc1_2.index]
    m+=1

In [50]:
fields = list(leads.columns)
fields = [e for e in fields if e not in ('Lead ID', 'Rating', 'Created Date', '18 Digit', 'Lead', 
                                         'Sale', 'fix', 'met', 'new_date', 'Mobile', 'Stage',
                                         'Lead Enquiry Stage', 'Count Activity', 'Branch',
                                         'Mobile Operator'
                                        )]

In [52]:
moncyc = mc
all_fields = {}
for field in fields:
    for i in range(8): 
        mon_cat = moncyc[i+1].groupby([moncyc[i+1][field]])
        if i == 0:
            mc_cat = pd.DataFrame(data=mon_cat['Sale'].sum()/mon_cat['Lead'].count()*100)
        else:
            mc_cat[str(i+1)] = mon_cat['Sale'].sum()/mon_cat['Lead'].count()*100
    mc_cat = mc_cat.rename(columns={0: 1})
    all_fields[field] = mc_cat

In [53]:
all_fields['Derived Age Group']

,1,2,3,4,5,6,7,8
Derived Age Group,,,,,,,,
27 to 30,0.120316,0.150015,0.224498,0.187328,0.215157,0.151603,0.153846,0.125112
31 to 35,0.221679,0.244260,0.322416,0.167740,0.219202,0.144244,0.222244,0.120100
36 to 39,0.396880,0.271955,0.316456,0.324957,0.205266,0.201235,0.220924,0.264224
40 to 49,0.514851,0.412123,0.452690,0.469925,0.397727,0.413724,0.427094,0.414376
age unknown,0.079807,0.080713,0.103288,0.096717,0.121126,0.098018,0.096582,0.083255
less than 27,0.072727,0.056148,0.058241,0.039502,0.022302,0.109625,0.049044,0.100553
more than 50,0.504673,0.404660,0.480384,0.558092,0.696244,0.569981,0.384911,0.485844


In [54]:
writer = pd.ExcelWriter('percentages_final.xlsx', engine='xlsxwriter')
for i in fields:
    all_fields[i].to_excel(writer, sheet_name=i)
writer.save()

In [57]:
writer = pd.ExcelWriter('field_count.xlsx', engine='xlsxwriter')
for field in fields:
    mon_cat = leads.groupby([field])
    p = pd.DataFrame(data=mon_cat['Lead'].count())
    p.to_excel(writer, sheet_name=field)
writer.save()